In [ ]:
import market
import pandas
from withdrawal import ConstantDollar, VPW
from harvesting import make_rebalancer
from portfolio import Portfolio
import metrics

In [ ]:
class CAPE10:
    def __init__(self):
        self.data = pandas.read_csv('cape10.csv')
        
    def to_row(self, year):
        first_year = self.data.iloc[0]['Year']
        row = year - first_year
        assert row >= 0, "Minimum year is %s" % first_year
        return int(row)

    def cape10(self, year):
        return self.data.iloc[self.to_row(year)]['CAPE10']
    def mean(self, year):
        return self.data.iloc[self.to_row(year)]['Mean']
        #return 16.7 # mean
    def median(self, year):
        return self.data.iloc[self.to_row(year)]['Median']
        #return 16.05 # median

In [ ]:
class CAPEHarvest:
    def __init__(self, cape, portfolio):
        self.cape = cape
        self.portfolio = portfolio

    def set_current_year(self, year):
        self.year = year
        
    def harvest(self):
        amount = yield
        while True:
            if cape.cape10(self.year) <= cape.median(self.year):
                first_chunk = min(self.portfolio.bonds, amount)
                self.portfolio.sell_bonds(first_chunk)
                self.portfolio.sell_stocks(amount - first_chunk)
            else:
                first_chunk = min(self.portfolio.stocks, amount)
                self.portfolio.sell_stocks(first_chunk)
                self.portfolio.sell_bonds(amount - first_chunk)
            amount = yield self.portfolio.empty_cash()          

In [ ]:
market_data = market.Returns_US_1871()
cape = CAPE10()

def simulate(series, year):
    portfolio = Portfolio(500000, 500000)
    
    years = 30

#    strategy = make_rebalancer(0.75)(portfolio).harvest()
    cape_strategy = CAPEHarvest(cape, portfolio)
    strategy = cape_strategy.harvest()
    cape_strategy.set_current_year(year)
    strategy.send(None)
    withdrawal_strategy = ConstantDollar(portfolio, strategy).withdrawals()
    annual = []

    # Withdrawals happen at the start of the year, so the first time
    # we don't have any performance data to send them....
    data = withdrawal_strategy.send(None)
    annual.append(data)
    year += 1

    for _, d in zip(range(years), series.iter_from(year)):
        cape_strategy.set_current_year(year)
        data = withdrawal_strategy.send(d)
        annual.append(data)
        year += 1
    return annual

pv = []
bonds = []
wd = []
count = 0
for i in range(1881, 1986):
    try:
        count += 1
        r = simulate(market_data, i)
        wd = wd + [n.withdraw_r for n in r]
        bonds.append(metrics.average([n.portfolio_bonds/n.portfolio_n for n in r]))
        pv.append(r[-1].portfolio_r)
    except:
        print('Failed', i)

print('Runs', count)
print("${:,}".format(int(metrics.median(pv))))
print(metrics.average(bonds))
print("${:,}".format(int(metrics.cew(wd))))